In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/contradictory-my-dear-watson/sample_submission.csv
/kaggle/input/contradictory-my-dear-watson/train.csv
/kaggle/input/contradictory-my-dear-watson/test.csv
/kaggle/input/watson-bert-baseline-training/__results__.html
/kaggle/input/watson-bert-baseline-training/watson-bert-base-multilingual-cased.h5
/kaggle/input/watson-bert-baseline-training/__notebook__.ipynb
/kaggle/input/watson-bert-baseline-training/__output__.json
/kaggle/input/watson-bert-baseline-training/custom.css


In [2]:
# ライブラリのインポート
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

In [3]:
# ハイパーパラメータの定義、トークナイザの初期化
model_name = "bert-base-multilingual-cased"
max_length = 120
batch_size = 16

tokenizer = AutoTokenizer.from_pretrained(model_name)

# 学習後のモデルの重みの読み込み
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
model.load_weights("../input/watson-bert-baseline-training/watson-bert-base-multilingual-cased.h5")

# テストデータの読み込み
test = pd.read_csv("../input/contradictory-my-dear-watson/test.csv")

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# 前提、仮説のテキストのみを抽出
test_text = test[['premise', 'hypothesis']].values.tolist()

# トークナイズ + インデキシング
test_encoded = tokenizer.batch_encode_plus(test_text, padding=True, max_length=120, truncation=True)

# TensorFlowのDatasetに変換
test_dataset = (
    tf.data.Dataset.from_tensor_slices((dict(test_encoded))).batch(batch_size)
)

# 推論
test_preds = model.predict(test_dataset)

# 推論したデータの確認。logitsがモデルの最終層の出力
test_preds

TFSequenceClassifierOutput(loss=None, logits=array([[-1.0137353 , -0.8597845 ,  1.630834  ],
       [-0.9524448 ,  1.3118819 , -0.31104618],
       [ 1.9467704 , -0.8014548 , -1.0568173 ],
       ...,
       [ 0.02177926,  0.8413682 , -0.81979316],
       [ 2.0388377 , -1.0136839 , -0.98619795],
       [-0.52115124, -1.086089  ,  1.4369633 ]], dtype=float32), hidden_states=None, attentions=None)

In [5]:
# 提出用ファイルの作成
submission = pd.read_csv("../input/contradictory-my-dear-watson/sample_submission.csv")
submission['prediction'] = test_preds["logits"].argmax(axis=1)
submission.to_csv('submission.csv', index=False)
submission.head()

,id,prediction
0,c6d58c3f69,2
1,cefcc82292,1
2,e98005252c,0
3,58518c10ba,1
4,c32b0d16df,2
